# **Magic**

**Hello guys, I hope you all had some nice kaggling and enjoyed this competition**

**Best of luck for all other competitions and congratz to all of you  for having this wonderful journey**

**As for the real reason you're here for, which is the ✨✨ M A G I C✨✨**

**Well enjoy this quick demonstration of one part of the magic we discovered 1 days before the competition ends**

**Make sure to upvote the notebook if you found it useful, it really helps me out 😃**

**Okay so this is how we discovered the magic**
**So, I knew that there was something fishy in the data so I visualized all the tweets and selected_text in the train dataset and kept on reading them until I found some interesting things**

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("../input/tweet-sentiment-extraction/train.csv")

**The next code snippet just prints the whole train tweets and selected_text 😅😅. I didn't run it here because it would slow down the notebook, you're free to remove the ( ''' ) at the beginning and end to run it**

In [ ]:
'''
for i,_ in train.iterrows():
    print("Text :")
    print(train.loc[i,"text"])
    print("Selected Text: ")
    print(train.loc[i,"selected_text"])
'''

**So, in all these different examples I didn't quite get anything except some very vague clues.**

# **Failures analysis**

**So, I used this failure dataset that has the majority of noise in the tweet dataset ( I downloaded this earlier and forgot where I got it from so if this belongs to you tell me so i can tag you here )**

In [ ]:
failures = pd.read_csv("../input/failures/failures.csv")

In [ ]:
failures.shape

**Hmm 3641 failures, this should narrow it down**

**Let's visualize this**

In [ ]:
failures.head()

**So, this is a  <span style="color:Red">MAJOR </span> part of the magic. As i visualized the selected_text and text in this failures dataset I noticed that there are no "spaces"**

**And boom there you got it.. Spaces. I knew there was something fishy with spaces so I went back looking at the training set**

**In here I merged the failures and the train in order to get the original text ( with the spaces )**

In [ ]:
fail = pd.merge(train, failures, how='inner', on=['textID'])

In [ ]:
fail.shape

In [ ]:
fail.head()

In [ ]:
'''
for i,_ in fail.iterrows():
    print("Text: ")
    print(fail.loc[i,"text_x"])
    print("Selected text: ")
    print(fail.loc[i,"selected_text_x"])
'''

**So, this is a key visualization, when going through all these tweets, I notice that there are so many spaces somtimes**

**Here's a trick you can do: Press CTRL + F to search for words and write "  " (that's a double space :p ) and then all the double spaces will be highlighted. Then, just scroll through the tweets above and see that double spaces are consistant**

# **Et voilà 🤩** 

**Let's count how many tweets with double spaces there is in the failures dataset**

In [ ]:
x=0
for i,row in fail.iterrows():
    if (fail.loc[i,"text_x"].count("  ")>0):
        x= x+1

In [ ]:
x

**Interesting: 2134.. Which is a lot ( more than half )**

# **🚀 Spaces Theory 🚀**

**Alright it's time to check if its really about the spaces**

In [ ]:
train = pd.read_csv("../input/tweet-sentiment-extraction/train.csv")

In [ ]:
train["Space"] = train["text"].str.count("  ")

In [ ]:
noise = train[train["Space"] > 0 ]

In [ ]:
noise.shape

**Wow O.O**

**Obviously here i'm gonna eliminate neutral tweets**

In [ ]:
noise = noise[noise["sentiment"]!="neutral"]

**In here i visualize all the tweets to check them out one by one ( Yeah this took a lot of time :'( )**

In [ ]:
'''
for i,row in noise.iterrows():
    print('Text :')
    print(noise.loc[i,"text"])
    print('Selected text :')
    print(noise.loc[i,"selected_text"])
'''

In [ ]:
fail.head()

In [ ]:
fail["Space"] = fail["text_x"].str.count("  ")

In [ ]:
fail = fail[fail["Space"] >0]

In [ ]:
fail.shape

In [ ]:
'''
for i,row in fail.iterrows():
    print('Text :')
    print(fail.loc[i,"text_x"])
    print('Selected text :')
    print(fail.loc[i,"selected_text_x"])
'''

**Alright so after days of looking at these examples I found a pattern:**

**Some kind of shift is happening**

**If you focus on the cases where the double space is before the selected_text, the selected_text gets the previous character allocated to it ( this also happens if we have a leading space )**

**So the obvious conclusion is that we need to add a certain number n of previous characters to the selected_text**

**That number n is the number of exceeding spaces before the selected_text**

**So for example if we have:**

**" This ‏‏‎ ‎is a test"**

**In here we have 1 exceeding space at the beginning + 1 exceeding space from the double space between "This" and "is"**

**So, if the selected_text was = "test" it would now be "a test"**

# **Creating a function that gets this noise to our data**

**This is an example tweet I took from the training**

In [ ]:
text =  " Haha I know, I cant handle the fame!  and thank you!"

In [ ]:
selected = "thank you!"

In [ ]:
#find how many spaces we have at the beginning ( this is pure exceeding spaces )
spaces_begin = len(text) - len(text.lstrip(' '))

In [ ]:
#get the index of the selected_text
i = text.find(selected)

**This function gets all the extra spaces on the left of our selected_text**

In [ ]:
extra_left = 0
spaces = 0
count = False
for j in range(spaces_begin,i):
    if (text[j] == ' ' and count == True):
        spaces = spaces+1
        
    if (text[j] == ' '):
        count = True
    else:
        count = False
        extra_left = extra_left + spaces
        spaces = 0

In [ ]:
extra_left

**This function gets all the extra spaces on the left of our selected_text**

In [ ]:
extra_right = 0
spaces = 0
count = False
for j in range(i,len(text)):
    if (text[j] == ' ' and count == True):
        spaces = spaces+1
        
    if (text[j] == ' '):
        count = True
    else:
        count = False
        extra_right = extra_right + spaces
        spaces = 0

In [ ]:
extra_right

**And now we create the new starting position**

In [ ]:
start = i-extra_left-spaces_begin

In [ ]:
text[start:i + len(selected)]

**Great! It works! It's just like the selected_text in the training set ( You can try this with many examples to understand it better )**

**Btw, I know i haven't used the extra_right value, its because still don't know that part of magic. From my observations, I didn't get a stable pattern from this so I won't be using it here.**

# **Adding ✨Magic✨ to your submission**

**So in here I'm gonan show you how I added this magic to my submission**

In [ ]:
sub = pd.read_csv("../input/sample-submission/submission (13).csv")

**Consider sub your submission**

In [ ]:
test = pd.read_csv("../input/tweet-sentiment-extraction/test.csv")

**We need to concatenate text and sentiment to our sub**

In [ ]:
sub["text"]= test["text"]
sub["sentiment"] = test["sentiment"]

**Rearrange the columns**

In [ ]:
cols= ['textID','text','selected_text','sentiment']
sub = sub[cols]

**Count the double spaces**

In [ ]:
sub["Space"] = sub["text"].str.count("  ")

**Stripping the selected_text ( as our model was always adding space in the beginning, this might not be the case for you)**

In [ ]:
sub["selected_text"] = sub["selected_text"].str.strip()

**Creating a df_noise that contains (almost) all the ouputs that should have noise**

In [ ]:
df_noise = sub[(sub["Space"] > 0) & ( sub["sentiment"] != "neutral")]

# **Now for the main algorithm**

In [ ]:
#Iterating all the should-be noise rows
for i, row in df_noise.iterrows():
    #Getting the original text
    text = df_noise.loc[i,"text"]
    #Getting the selected_text
    selected = df_noise.loc[i,"selected_text"]
    
    #First extra spaces
    spaces_begin = len(text) - len(text.lstrip(' '))
    #Finding the selected_text in Text
    idx = text.find(selected)
    #Now we sometimes have the double space inside the selected_text which would give us idx = -1
    #And thus, gives us selected_text= nan
    #This will hinder your score. I did this mistake when I first tried this and almost lost hope if I hadn't visualized the outputs
    if ( idx != -1):
        #left spaces
        extra_left = 0
        spaces = 0
        count = False
        for j in range(spaces_begin,idx):
            if (text[j] == ' ' and count == True):
                spaces = spaces+1

            if (text[j] == ' '):
                count = True
            else:
                count = False
                extra_left = extra_left + spaces
                spaces = 0
        #right spaces (unneeded here)
        extra_right = 0
        spaces = 0
        count = False
        for j in range(idx,len(text)):
            if (text[j] == ' ' and count == True):
                spaces = spaces+1

            if (text[j] == ' '):
                count = True
            else:
                count = False
                extra_right = extra_right + spaces
                spaces = 0
        start = idx-extra_left-spaces_begin
        #Creating the new selected_text
        new_selected = text[start:idx + len(selected)]
    else:
        #If idx=-1, I just kep the selected_text without noise
        new_selected = selected

    df_noise.loc[i,"selected_text"] = new_selected.strip()

**Adding these to your submission**

In [ ]:
for i, row in df_noise.iterrows():
    sub.loc[i,"selected_text"] = df_noise.loc[i,"selected_text"]

In [ ]:
#Dropping unecessary columns
sub = sub.drop(["text","sentiment","Space"], axis=1)

In [ ]:
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)

# **And there it is boys.**
**This simple yet effective change gave us 0.005 in LB**

**Although this works, I'm pretty sure there are other improvements ( other magic tricks ) that can improve the score**

**I hope this helped some of you. If there are any improvements or any remarks please comment down below and don't forget to upvote it really helps me out a bunch**

**Thank you all for your interest and time and I hope you have a great day !**